In [1]:
import pickle

from dataset import NewsDataset
from model import DistilBertForSequenceClassification

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

import torch
from torch import nn
from transformers import DistilBertConfig, DistilBertTokenizer

from IPython.display import display, HTML

In [2]:
config = DistilBertConfig()
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distilbert = DistilBertForSequenceClassification(config, num_labels=93)

criterion = nn.CrossEntropyLoss()

path = '/media/vitaliy/9C690A1791D68B8B/after/learningfolder/distilbert_medium_titles/distilbert.pth'

if torch.cuda.is_available():
    distilbert.load_state_dict(
        torch.load(path)
    )
else:
    distilbert.load_state_dict(
        torch.load(path, map_location=torch.device('cpu'))
    )
    
with open('../label_encoder.sklrn', 'rb') as f:
    le = pickle.load(f)

/home/vitaliy/anaconda3/envs/sqrt/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.2.post1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
test_example = [
    ["Interpretation of HuggingFase's model decision"], 
    ["Transformer-based models have taken a leading role"
     "in NLP today."]
]

test_dataset = NewsDataset(
    data_list=test_example,
    tokenizer=tokenizer,
    max_length=config.max_position_embeddings, 
    one_example=True
)

In [ ]:
# smooth_grad = SmoothGradient(distilbert, criterion)
# instances = smooth_grad.saliency_interpret(test_dataset)
integrated_grads = IntegratedGradient(distilbert, criterion)
instances = integrated_grads.saliency_interpret(test_dataset)

> /home/vitaliy/vitaliy/workfolder/repos/Interpret/integrated_gradient.py(96)_integrate_gradients()
-> embeddings_list.reverse()
(Pdb) type(grads)
<class 'dict'>


In [ ]:
coloder_string = smooth_grad.colorize(instances['instance_1'])
display(HTML(coloder_string))

In [ ]:
label = instances['instance_1']['label_input_1']
print(f"Converted label #{label}: {le.inverse_transform([label])[0]}")